In [2]:
import platform

platform.python_version()

'3.11.3'

## 1. 스레드 처리하기 

- 파이썬에서 스레드를 사용하여 간단한 동시성 처리를 구현하는 예제를 제공합니다. 
- 스레드는 파이썬의 threading 모듈을 통해 사용할 수 있습니다.

In [5]:
import threading
import time

In [8]:
for i in dir(threading) :
    if not i.startswith("_") : 
        print(i, end=", ")

Barrier, BoundedSemaphore, BrokenBarrierError, Condition, Event, ExceptHookArgs, Lock, RLock, Semaphore, TIMEOUT_MAX, Thread, ThreadError, Timer, WeakSet, activeCount, active_count, currentThread, current_thread, enumerate, excepthook, functools, get_ident, get_native_id, getprofile, gettrace, local, main_thread, setprofile, settrace, stack_size, 

##  2. 멀티 스레드 처리 

- threading.Thread를 사용하여 각 작업을 별도의 스레드로 만들고, start() 메서드로 각 스레드를 시작합니다. 
- 그리고 join() 메서드로 각 스레드가 완료될 때까지 기다립니다.

### 두 개의 작업(task1 및 task2)을 각각의 스레드에서 실행합니다.

In [18]:
def task1():
    for _ in range(5):
        print("Task 1 is running  ",threading.current_thread()," ")
        time.sleep(1)

def task2():
    for _ in range(5):
        print("Task 2 is running  ",threading.current_thread(), " ")
        time.sleep(1)

###  스레드를 2개 만들어서 실행합니다.

In [19]:
if __name__ == "__main__":
    # 두 개의 스레드 생성
    thread1 = threading.Thread(target=task1)
    thread2 = threading.Thread(target=task2)
    
    print("main thread : ", threading.main_thread())
    # 각각의 스레드 시작
    thread1.start()
    thread2.start()
    print("active count : ", threading.active_count())
    
    # 각각의 스레드가 완료될 때까지 대기
    thread1.join()
    thread2.join()

    print("Both tasks are completed")


main thread :  <_MainThread(MainThread, started 7978636032)>
Task 1 is running   <Thread(Thread-21 (task1), started 6409170944)>  
Task 2 is running   <Thread(Thread-22 (task2), started 6425997312)>  
active count :  10
Task 1 is running  Task 2 is running   <Thread(Thread-22 (task2), started 6425997312)>  
 <Thread(Thread-21 (task1), started 6409170944)>  
Task 2 is running  Task 1 is running   <Thread(Thread-21 (task1), started 6409170944)>  
 <Thread(Thread-22 (task2), started 6425997312)>  
Task 2 is running   <Thread(Thread-22 (task2), started 6425997312)>  
Task 1 is running   <Thread(Thread-21 (task1), started 6409170944)>  
Task 2 is running   <Thread(Thread-22 (task2), started 6425997312)>  
Task 1 is running   <Thread(Thread-21 (task1), started 6409170944)>  
Both tasks are completed


## 3. 스레드간 데이터 공유

- 스레드 간에 데이터를 안전하게 전달하기 위해서는 threading 모듈에서 제공하는 Lock 및 Queue와 같은 도구를 사용할 수 있습니다. 
- 여러 스레드에서 공유되는 데이터에 대한 안전성을 보장하기 위해 동기화 메커니즘이 필요합니다.

- threading.Lock을 사용하여 데이터에 대한 접근을 동기화하고, queue.Queue를 사용하여 스레드 간 데이터 전달을 구현합니다.

In [20]:
import queue

In [21]:
def producer(queue, lock):
    for i in range(5):
        time.sleep(1)
        with lock:
            print(f"Producing {i}")
            # 큐에 정보를 전달 
            queue.put(i)

def consumer(queue, lock):
    while True:
        time.sleep(1)
        with lock:
            if not queue.empty():
                # 큐에 정보를 처리
                item = queue.get()
                print(f"Consuming {item}")
            else:
                break

In [22]:

if __name__ == "__main__":
    
    # 큐 생성
    shared_queue = queue.Queue()
    
    # 스레드 락 
    shared_lock = threading.Lock()

    # Producer 스레드 시작
    producer_thread = threading.Thread(target=producer, args=(shared_queue, shared_lock))
    producer_thread.start()

    # Consumer 스레드 시작
    consumer_thread = threading.Thread(target=consumer, args=(shared_queue, shared_lock))
    consumer_thread.start()

    # 각 스레드가 완료될 때까지 대기
    producer_thread.join()
    consumer_thread.join()

    print("Both threads are completed")


Producing 0
Consuming 0
Producing 1
Consuming 1
Producing 2
Consuming 2
Producing 3
Consuming 3
Producing 4
Both threads are completed


## 4. 뮤텍스 처리 


- 스레드 간의 데이터를 안전하게 동기화하기 위해 뮤텍스(뮤텐)를 사용할 수 있습니다. 
- 뮤텍스는 락(lock)이라고도 불리며, 여러 스레드가 동시에 공유 데이터에 접근하는 것을 막기 위해 사용됩니다.

- 파이썬에서는 threading 모듈의 Lock 클래스를 사용하여 뮤텍스를 구현할 수 있습니다. 
- 뮤텍스를 사용하면 특정 코드 블록을 하나의 스레드만 실행할 수 있도록 만들 수 있습니다.

### 뮤텍스 처리 

- threading.Lock을 사용하여 modify_shared_data 함수에서 공유 데이터 shared_data를 안전하게 수정하고 있습니다. 

- with mutex: 구문을 사용하여 뮤텍스가 활성화된 동안에만 해당 코드 블록이 실행되도록 하였습니다.

In [25]:
# 공유 데이터
shared_data = 0

# 뮤텍스 생성
mutex = threading.Lock()

def modify_shared_data():
    global shared_data
    with mutex:
        # 뮤텍스를 사용하여 공유 데이터 안전하게 수정
        for _ in range(1000):
            shared_data += 1

In [26]:

if __name__ == "__main__":
    # 두 개의 스레드 생성
    thread1 = threading.Thread(target=modify_shared_data)
    thread2 = threading.Thread(target=modify_shared_data)

    # 각각의 스레드 시작
    thread1.start()
    thread2.start()

    # 각각의 스레드가 완료될 때까지 대기
    thread1.join()
    thread2.join()

    print("Final shared_data:", shared_data)


Final shared_data: 2000
